In [2]:
!pip install transformers torch datasets


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math

In [4]:
model_name = "readerbench/RoBERT-base"  # Romanian RoBERTa

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/245k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/463M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at readerbench/RoBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(37788, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
def compute_metrics(model, tokenizer, text, verbose=False):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    neg_log_likelihood = -torch.mean(torch.log(probabilities.max(dim=-1).values + 1e-9))

    # Only print debugging info if verbose=True
    if verbose:
        print(f"Text: {text[:50]}... | NLL: {neg_log_likelihood:.4f}")

    return neg_log_likelihood.item(), neg_log_likelihood.item()


In [7]:
import json
from tqdm import tqdm

In [8]:
regions = [
    'Ardeal',
    'Banat',
    'Bucovina',
    'Canada_EN',
    'Canada_Quebec',
    'Crisana',
    'Dobrogea',
    'Germania',
    'Italia',
    'Maramures',
    'Moldova',
    'Muntenia',
    'Oltenia',
    'Serbia',
    'Spania',
    'Ucraina',
    'UK'
]

In [9]:
import json
from tqdm import tqdm
import numpy as np

with open("/kaggle/input/dataset-tari/dataset/Italia.json") as f:
    italia = json.load(f)

In [18]:
import json
import numpy as np
from tqdm import tqdm

results_regions = {}

for region in regions:
    results_regions[region] = []
    results_content = {"perplexity": [], "neg_log_likelihood": []}
    results_titles = {"perplexity": [], "neg_log_likelihood": []}

    with open(f"/kaggle/input/dataset-tari/dataset/{region}.json") as f:
        region_json = json.load(f)

    for row in tqdm(region_json, disable=True):  # Disable progress bar output
        row_cnt_result = compute_metrics(model, tokenizer, row['content'] if 'content' in row else row['text'], verbose=False)
        results_content['perplexity'].append(row_cnt_result[0])
        results_content['neg_log_likelihood'].append(row_cnt_result[1])
        
        row_title_result = compute_metrics(model, tokenizer, row['title'], verbose=False)
        results_titles['perplexity'].append(row_title_result[0])
        results_titles['neg_log_likelihood'].append(row_title_result[1])

    perp_content_mean = np.array(results_content['neg_log_likelihood']).mean()
    perp_titles_mean = np.array(results_titles['neg_log_likelihood']).mean()

    results_regions[region].append({
        'content': results_content,
        'titles': results_titles,
        'perp_mean_content': perp_content_mean,
        'perp_mean_titles': perp_titles_mean
    })


In [27]:
import json
import numpy as np

# Load results
with open("/kaggle/working/results_regions.json", "r", encoding="utf-8") as f:
    results_regions = json.load(f)

# Store all perplexities for overall statistics
all_perplexities_content = []
all_perplexities_titles = []

print("\n📊 **Perplexity Statistics Per Region:**")
for region, data in results_regions.items():
    # Extract **all** perplexities, not just means
    region_perplexities_content = [p for entry in data for p in entry["content"]["perplexity"]]
    region_perplexities_titles = [p for entry in data for p in entry["titles"]["perplexity"]]

    if not region_perplexities_content or not region_perplexities_titles:
        print(f"⚠️ No data available for {region}. Skipping...")
        continue  # Avoid crashing if a region has no perplexities

    # Compute mean, min, and max correctly from all values
    mean_content = np.mean(region_perplexities_content)
    min_content = np.min(region_perplexities_content)
    max_content = np.max(region_perplexities_content)

    mean_titles = np.mean(region_perplexities_titles)
    min_titles = np.min(region_perplexities_titles)
    max_titles = np.max(region_perplexities_titles)

    # Print per-region statistics
    print(f"📍 {region}:")
    print(f"   - **Content**: Mean = {mean_content:.4f}, Min = {min_content:.4f}, Max = {max_content:.4f}")
    print(f"   - **Titles**:  Mean = {mean_titles:.4f}, Min = {min_titles:.4f}, Max = {max_titles:.4f}\n")

    # Collect for overall dataset statistics
    all_perplexities_content.extend(region_perplexities_content)
    all_perplexities_titles.extend(region_perplexities_titles)

# Compute overall dataset statistics
if all_perplexities_content and all_perplexities_titles:
    dataset_mean_content = np.mean(all_perplexities_content)
    dataset_min_content = np.min(all_perplexities_content)
    dataset_max_content = np.max(all_perplexities_content)

    dataset_mean_titles = np.mean(all_perplexities_titles)
    dataset_min_titles = np.min(all_perplexities_titles)
    dataset_max_titles = np.max(all_perplexities_titles)

    # Print overall dataset statistics
    print("\n📊 **Overall Dataset Perplexity Statistics:**")
    print(f"   - **Content**: Mean = {dataset_mean_content:.4f}, Min = {dataset_min_content:.4f}, Max = {dataset_max_content:.4f}")
    print(f"   - **Titles**:  Mean = {dataset_mean_titles:.4f}, Min = {dataset_min_titles:.4f}, Max = {dataset_max_titles:.4f}")
else:
    print("⚠️ No perplexity data found in the dataset.")



📊 **Perplexity Statistics Per Region:**
📍 Ardeal:
   - **Content**: Mean = 0.5664, Min = 0.3680, Max = 0.6930
   - **Titles**:  Mean = 0.6265, Min = 0.4251, Max = 0.6931

📍 Banat:
   - **Content**: Mean = 0.5776, Min = 0.4119, Max = 0.6927
   - **Titles**:  Mean = 0.6198, Min = 0.3661, Max = 0.6931

📍 Bucovina:
   - **Content**: Mean = 0.5715, Min = 0.3950, Max = 0.6930
   - **Titles**:  Mean = 0.6249, Min = 0.4518, Max = 0.6927

📍 Canada_EN:
   - **Content**: Mean = 0.5977, Min = 0.4460, Max = 0.6918
   - **Titles**:  Mean = 0.6523, Min = 0.5727, Max = 0.6930

📍 Canada_Quebec:
   - **Content**: Mean = 0.6069, Min = 0.4089, Max = 0.6912
   - **Titles**:  Mean = 0.6568, Min = 0.5344, Max = 0.6928

📍 Crisana:
   - **Content**: Mean = 0.5996, Min = 0.3765, Max = 0.6928
   - **Titles**:  Mean = 0.6298, Min = 0.4546, Max = 0.6927

📍 Dobrogea:
   - **Content**: Mean = 0.5785, Min = 0.3476, Max = 0.6928
   - **Titles**:  Mean = 0.6153, Min = 0.4242, Max = 0.6927

📍 Germania:
   - **Content**

In [26]:
import json
import numpy as np

# Load results
with open("/kaggle/working/results_regions.json", "r", encoding="utf-8") as f:
    results_regions = json.load(f)

# Store all perplexities for overall dataset statistics
all_perplexities_content = []
all_perplexities_titles = []

print("\n📊 **Perplexity Min & Max Per Region:**")
for region, data in results_regions.items():
    # Extract **all** perplexities, not just mean values
    region_perplexities_content = [p for entry in data for p in entry["content"]["perplexity"]]
    region_perplexities_titles = [p for entry in data for p in entry["titles"]["perplexity"]]

    if not region_perplexities_content or not region_perplexities_titles:
        print(f"⚠️ No data available for {region}. Skipping...")
        continue  # Avoid crashing if a region has no perplexities

    # Compute min & max correctly from all values
    min_content = np.min(region_perplexities_content)
    max_content = np.max(region_perplexities_content)
    min_titles = np.min(region_perplexities_titles)
    max_titles = np.max(region_perplexities_titles)

    # Print per-region min & max
    print(f"📍 {region}:")
    print(f"   - Min Perplexity (Content): {min_content:.4f}")
    print(f"   - Max Perplexity (Content): {max_content:.4f}")
    print(f"   - Min Perplexity (Titles): {min_titles:.4f}")
    print(f"   - Max Perplexity (Titles): {max_titles:.4f}\n")

    # Collect for overall dataset stats
    all_perplexities_content.extend(region_perplexities_content)
    all_perplexities_titles.extend(region_perplexities_titles)

# Compute overall dataset min & max
if all_perplexities_content and all_perplexities_titles:
    dataset_min_content = np.min(all_perplexities_content)
    dataset_max_content = np.max(all_perplexities_content)
    dataset_min_titles = np.min(all_perplexities_titles)
    dataset_max_titles = np.max(all_perplexities_titles)

    # Print overall dataset min & max
    print("\n📊 **Overall Dataset Perplexity Min & Max:**")
    print(f"   - Min Perplexity (Content): {dataset_min_content:.4f}")
    print(f"   - Max Perplexity (Content): {dataset_max_content:.4f}")
    print(f"   - Min Perplexity (Titles): {dataset_min_titles:.4f}")
    print(f"   - Max Perplexity (Titles): {dataset_max_titles:.4f}")
else:
    print("⚠️ No perplexity data found in the dataset.")



📊 **Perplexity Min & Max Per Region:**
📍 Ardeal:
   - Min Perplexity (Content): 0.3680
   - Max Perplexity (Content): 0.6930
   - Min Perplexity (Titles): 0.4251
   - Max Perplexity (Titles): 0.6931

📍 Banat:
   - Min Perplexity (Content): 0.4119
   - Max Perplexity (Content): 0.6927
   - Min Perplexity (Titles): 0.3661
   - Max Perplexity (Titles): 0.6931

📍 Bucovina:
   - Min Perplexity (Content): 0.3950
   - Max Perplexity (Content): 0.6930
   - Min Perplexity (Titles): 0.4518
   - Max Perplexity (Titles): 0.6927

📍 Canada_EN:
   - Min Perplexity (Content): 0.4460
   - Max Perplexity (Content): 0.6918
   - Min Perplexity (Titles): 0.5727
   - Max Perplexity (Titles): 0.6930

📍 Canada_Quebec:
   - Min Perplexity (Content): 0.4089
   - Max Perplexity (Content): 0.6912
   - Min Perplexity (Titles): 0.5344
   - Max Perplexity (Titles): 0.6928

📍 Crisana:
   - Min Perplexity (Content): 0.3765
   - Max Perplexity (Content): 0.6928
   - Min Perplexity (Titles): 0.4546
   - Max Perplexity 